## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-21-09-00-06 +0000,bbc,UK homes to get £15bn for solar and green tech...,https://www.bbc.com/news/articles/ckgj7me00p0o...
1,2026-01-21-08-58-00 +0000,wsj,Burberry Shares Pop as Luxury Trench-Coat Make...,https://www.wsj.com/business/retail/burberry-b...
2,2026-01-21-08-57-00 +0000,wsj,Sterling’s Rise After Inflation Data Is Unlike...,https://www.wsj.com/finance/currencies/yen-con...
3,2026-01-21-08-52-06 +0000,nyt,Air Force One Turns Back With Trump After Elec...,https://www.nytimes.com/2026/01/20/us/politics...
4,2026-01-21-08-44-34 +0000,bbc,Life sentence for man who killed Japan's ex-PM...,https://www.bbc.com/news/articles/c86v5lj39zpo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2366/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
28,trump,98
70,greenland,41
94,new,19
339,court,16
145,davos,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
361,2026-01-20-10-36-28 +0000,bbc,"Greenland 'will stay Greenland', former Trump ...",https://www.bbc.com/news/articles/c98p6m0r53no...,202
95,2026-01-21-01-05-00 +0000,wsj,President Trump’s campaign for Greenland follo...,https://www.wsj.com/politics/policy/trumps-gre...,184
56,2026-01-21-03-26-41 +0000,nypost,Trump teases how far he’ll go to acquire Green...,https://nypost.com/2026/01/20/us-news/trump-te...,183
170,2026-01-20-22-10-25 +0000,wapo,Stocks slide after Trump threatens new tariffs...,https://www.washingtonpost.com/business/2026/0...,179
364,2026-01-20-10-30-00 +0000,wsj,Wednesday’s Supreme Court arguments over wheth...,https://www.wsj.com/economy/central-banking/je...,178


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
361,202,2026-01-20-10-36-28 +0000,bbc,"Greenland 'will stay Greenland', former Trump ...",https://www.bbc.com/news/articles/c98p6m0r53no...
364,69,2026-01-20-10-30-00 +0000,wsj,Wednesday’s Supreme Court arguments over wheth...,https://www.wsj.com/economy/central-banking/je...
340,64,2026-01-20-12-09-26 +0000,cbc,EU chief promises 'unflinching' response to ne...,https://www.cbc.ca/news/world/greenland-europe...
304,58,2026-01-20-15-58-06 +0000,nypost,Trump tells The Post why he posted Emmanuel Ma...,https://nypost.com/2026/01/20/us-news/trump-re...
90,57,2026-01-21-01-17-00 +0000,wsj,Escalating tensions over Greenland are superch...,https://www.wsj.com/economy/global/united-stat...
45,52,2026-01-21-04-15-28 +0000,nypost,Air Force One forced to turn around after Trum...,https://nypost.com/2026/01/20/us-news/air-forc...
6,46,2026-01-21-08-42-08 +0000,nyt,Photographer Discovers Train Part That Could H...,https://www.nytimes.com/2026/01/20/world/europ...
168,45,2026-01-20-22-12-28 +0000,nyt,Minnesota Police Leaders Criticize Federal Tac...,https://www.nytimes.com/2026/01/20/us/ice-minn...
312,44,2026-01-20-15-15-07 +0000,latimes,Netflix amends Warner Bros. deal to all cash i...,https://www.latimes.com/entertainment-arts/bus...
301,40,2026-01-20-16-06-20 +0000,nypost,Treasury Secretary Scott Bessent lashes out at...,https://nypost.com/2026/01/20/business/trumps-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
